<a href="https://colab.research.google.com/github/nagavarma9199/neural-networks/blob/main/ICP_9_1%262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D


In [2]:
import pandas as pd


dataset = pd.read_csv('Sentiment.csv', header=0)

mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]


In [3]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-3-cee1da567eb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-3-cee1da567eb8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [4]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

In [5]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [6]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [8]:
embed_dim = 128
lstm_out = 196


In [9]:
def create_model():
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()

In [10]:
batch_size = 32
model.fit(X_train, Y_train, epochs=10, batch_size=batch_size, verbose=2)

# Save the model
model.save('sentiment_model.h5')

Epoch 1/10
230/230 - 33s - loss: 0.8580 - accuracy: 0.6255 - 33s/epoch - 145ms/step
Epoch 2/10
230/230 - 21s - loss: 0.7128 - accuracy: 0.7027 - 21s/epoch - 91ms/step
Epoch 3/10
230/230 - 21s - loss: 0.6259 - accuracy: 0.7370 - 21s/epoch - 90ms/step
Epoch 4/10
230/230 - 21s - loss: 0.5715 - accuracy: 0.7638 - 21s/epoch - 90ms/step
Epoch 5/10
230/230 - 19s - loss: 0.5173 - accuracy: 0.7881 - 19s/epoch - 83ms/step
Epoch 6/10
230/230 - 22s - loss: 0.4699 - accuracy: 0.8079 - 22s/epoch - 94ms/step
Epoch 7/10
230/230 - 20s - loss: 0.4152 - accuracy: 0.8326 - 20s/epoch - 86ms/step
Epoch 8/10
230/230 - 21s - loss: 0.3739 - accuracy: 0.8473 - 21s/epoch - 92ms/step
Epoch 9/10
230/230 - 21s - loss: 0.3423 - accuracy: 0.8618 - 21s/epoch - 90ms/step
Epoch 10/10
230/230 - 22s - loss: 0.3124 - accuracy: 0.8744 - 22s/epoch - 94ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1 ) Save the model and use the saved model to predict on new text data (ex, “A lot of good things are
happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [20]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [21]:
model.save('sentimentAnalysis.h5') #Saving the model

In [22]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [23]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 1 0 0]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
10936    Positive
10937    Negative
10938     Neutral
10939    Negative
10940    Negative
Name: sentiment, Length: 10941, dtype: object


In [24]:
def predict_sentiment(text, tokenizer, model, max_features):
    text = text.lower()
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    text = text.replace('rt', ' ')
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_features)
    prediction = model.predict(padded_sequence)
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    return sentiment_labels[np.argmax(prediction)]

In [25]:
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"


In [26]:
predicted_sentiment = predict_sentiment(new_text, tokenizer, model, X.shape[1])
print("Predicted Sentiment:", predicted_sentiment)

1/1 [==============================] - 0s 220ms/step
Predicted Sentiment: Positive


Apply GridSearchCV on the source code provided in the class

In [ ]:
pip install scikeras

In [ ]:
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


744/744 - 171s - loss: 0.8226 - accuracy: 0.6460 - 171s/epoch - 230ms/step
186/186 - 5s - 5s/epoch - 29ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


744/744 - 134s - loss: 0.8190 - accuracy: 0.6477 - 134s/epoch - 180ms/step
186/186 - 5s - 5s/epoch - 26ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/2
186/186 - 45s - loss: 0.8454 - accuracy: 0.6354 - 45s/epoch - 244ms/step
Epoch 2/2
186/186 - 41s - loss: 0.6908 - accuracy: 0.7040 - 41s/epoch - 222ms/step
47/47 - 1s - 1s/epoch - 29ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/2
186/186 - 47s - loss: 0.8393 - accuracy: 0.6343 - 47s/epoch - 254ms/step
Epoch 2/2
186/186 - 40s - loss: 0.6784 - accuracy: 0.7111 - 40s/epoch - 215ms/step
47/47 - 2s - 2s/epoch - 52ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/2
465/465 - 87s - loss: 0.8202 - accuracy: 0.6494 - 87s/epoch - 186ms/step
Epoch 2/2
465/465 - 82s - loss: 0.6787 - accuracy: 0.7165 - 82s/epoch - 176ms/step
Best: 0.684170 using {'batch_size': 20, 'epochs': 2}
